In [1]:
import sys, os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
sys.path.insert(0, os.path.dirname(os.getcwd()))

In [2]:
from audio_utils import SpecViewer
import librosa
import numpy as np
from glob import glob
import json
from model import MultiChannelWhisperSeg

In [3]:
mc_segmenter = MultiChannelWhisperSeg( "nccratliri/mc-whisperseg-zebra-finch-ct2-v1.0", device="cuda" )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
spec_viewer = SpecViewer()

In [5]:
sr = 16000
base_fname = "../data/example_subset/Zebra_finch/test/BP_2021-05-28_15-35-42_115696_0280000"

radio_fname_list = [ 
     base_fname + "_radio1.wav",
     base_fname + "_radio2.wav"
]
mic_fname_list = [
     base_fname + "_daq1.wav"
]

In [6]:
radio_channels = [ librosa.load( fname, sr = sr )[0] for fname in radio_fname_list ]
mic_channels = [ librosa.load( fname, sr = sr )[0] for fname in mic_fname_list ]

In [7]:
label = json.load( open( base_fname + "_radio1.json" ) )

In [8]:
spec_viewer.visualize(
    np.flip(np.asarray([ radio_channels[0], radio_channels[1], mic_channels[0] ]), axis = 0),
    sr = sr,
    label=label,
    window_size = 2.5
)

interactive(children=(FloatSlider(value=208.375, description='offset', max=416.9304375, step=0.125), Output())…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [9]:
predictions = mc_segmenter.segment( radio_channels, mic_channels, sr,
                      min_frequency = 0,
                      spec_time_step = 0.0025,
                      min_segment_length = 0.005,
                      eps = 0.02,
                      num_trials = 3
                    )

Segmenting radio channel 0 [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00%
Segmenting radio channel 1 [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00%


In [10]:
spec_viewer.visualize(
    np.flip(np.asarray([ radio_channels[0], radio_channels[1], mic_channels[0] ]), axis = 0),
    sr = sr,
    label=label,
    prediction=predictions[0],
    window_size = 2.5
)

interactive(children=(FloatSlider(value=208.375, description='offset', max=416.9304375, step=0.125), Output())…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>